<a href="https://colab.research.google.com/github/siting1206/NLP_HW2/blob/main/BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim import corpora
from gensim.summarization import bm25
from nltk.stem.porter import PorterStemmer  
from collections import namedtuple
import regex as re
import heapq
p_stemmer = PorterStemmer()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
Tag = namedtuple("Tag", ["snippets", "question", "answer"])

# Load train/validation dataset definition
def load_sequences(filename, sep="\t", notypes=False, test_data=False):
    sequences = []
    with open(filename) as fp:
        seq = []
        for line in fp:
            line = line.strip()
            if line:
                line = line.replace(" ||| ", "\t")
                line = line.split(sep)
                seq.append(Tag(*line))
            else:
                sequences.append(seq)
                seq = []
        if seq:
            sequences.append(seq)
    return sequences

In [4]:
train_sequences = load_sequences("drive/MyDrive/NLP_assignment2/Dataset/train.txt", sep="\t", notypes=True)

In [15]:
train_snippets = [[t[0] for t in seq] for seq in (train_sequences)] # length = 99820
train_question = [[t[1] for t in seq] for seq in (train_sequences)]
train_answer = [[t[2] for t in seq] for seq in (train_sequences)]

In [16]:
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

In [17]:
for i in range(0, len(train_snippets[0])):
  train_snippets[0][i] = re.findall('<s> (.*?) </s>', train_snippets[0][i])
  train_answer[0][i] = train_answer[0][i].split()

In [18]:
article_list =[]
for a in train_snippets[0][0]:
    a_split = a.replace('?',' ').replace('(',' ').replace(')',' ').split(' ')
    # 詞干提取(e.g. assignment->assign, words->word)
    stemmed_tokens = [p_stemmer.stem(i) for i in a_split]  
    article_list.append(stemmed_tokens)

print(article_list)
query =[train_answer[0][0]]
print(query[0])
query_stemmed = [p_stemmer.stem(i) for i in query[0]]  
print('query_stemmed :',query_stemmed )

# bm25模型
bm25Model = bm25.BM25(article_list)
# 逆文件頻率
average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
scores = bm25Model.get_scores(query_stemmed,average_idf)
print('scores :',scores)
max_index = sort_index(scores)[:3]
trn_s = []
for i in range(0, len(max_index)):
  n = max_index[i]
  trn_s.append(train_snippets[0][0][n])
# print(trn_s)

[['assign', '13', 'nov', '30', ',', '2014', '3', 'letter', 'word', '200', 'question', '1', 'last', '8', 'year', 'life', ',', 'galileo', 'hous', 'arrest', 'espous', 'man', "'s", 'theori', '2'], ['galileo', 'affair', 'wikipedia', 'galileo', 'affair', 'sequenc', 'event', ',', 'begin', 'around', '1610', ',', 'culmin', 'trial', '1632', 'galileo', ',', 'old', 'man', ',', 'publish', 'dialogu', 'concern', 'two', 'chief', 'galileo', 'kept', 'hous', 'arrest', 'death', '1642', 'heliocentr', ',', 'theori', 'earth', 'planet', ',', 'along'], ['galileo', 'get', 'troubl', 'right', ',', 'jerk', 'io9', 'sep', '15', ',', '2011', 'galileo', 'face', 'stiff', 'odd', 'publish', 'dialogu', 'concern', 'galileo', 'spend', 'last', 'year', 'life', 'hous', 'arrest', 'fair', 'mind', ',', 'formal', 'grant', 'galileo', 'write', 'theori', 'one', 'espous', 'aristotelian', 'geocentr', 'view', '8', '23', '17', '6k'], ['web', 'navig', 'last', '8', 'year', 'life', ',', 'galileo', 'hous', 'arrest', 'espous', 'man', "'s", 't